# Crawling_assignment1_Tobigs14_이혜린

# 네이버 자주 본 뉴스 크롤링

1) 수집내용

    1) 많이본뉴스–섹션별(정치~IT/과학)Top5기사제목,신문사,뷰 
    2) 해당 기사별 기사 내용, 리액션 (좋아요 ~ 후속기사 원해요)
    
2) 수집방법(택1)

    1) [기본] Requests , BeautifulSoup, Selenium
    2) [심화] Requests, BeautifulSoup (+ 멀티프로세싱)
    
3) 수집범위 및 저장

    1) 2019년7월21일~2020년8월20일(동작가능,실제구동x)
    2) 하나의 파일로 저장 (방식 자유)
    3) Ex)총6섹션*Top5*365일=10950rows

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

from IPython.display import Image
import os
from pathlib import Path
import glob

import datetime

## 날짜 String 생성

In [2]:
start_date = datetime.datetime(2019, 7, 21, 3, 22, 32)
# start_date.strftime("%Y%m%d %Y-%m-%d 시간 표시외 %H:%M:%S 아무거나 넣어도 됩니다.") #strftime 활용 예시

In [3]:
def get_date_string(start_date, period=365):
    # 20190721 형태의 Date String List 생성
    return [
        (start_date + datetime.timedelta(days=day)).strftime("%Y%m%d")
        for day in range(period)
    ]

In [4]:
date_string_list = get_date_string(start_date, 366)

In [5]:
date_string_list[0], date_string_list[-1] # 생성 완료

('20190721', '20200720')

## Part 1. Request를 이용한 크롤링

In [6]:
def get_top_news(date):
    """
    해당 날짜의 자주 본 뉴스 25개에 대한 정보를 반환하는 함수입니다.
    """
    url = f"https://news.naver.com/main/ranking/popularDay.nhn?rankingType=popular_day&date={date}" 
    # 이부분을 잘 채워넣어주세요 (네이버 뉴스 자주보는 뉴스 링크에서 date를 제외한 부분)
    
    res = requests.get(url)
    soup = BeautifulSoup(res.text, "html.parser", from_encoding='euc-kr')

    # 1. 신문 제목
    titles_html = soup.select('.ranking_section > ol > li > dl > dt > a')

    # 2. 기사 링크
    link_list = []
    for a in soup.select("li > dl > dt > a "):
        link_list.append(a['href'])

    # 3. 신문사명
    name_list = []
    for dl in soup.select("li > dl"):
        name_list.append(dl.select('span')[-2].text.strip())

    # 4. View
    view_html = soup.select('li i')
    
    result = []

    for i in range(len(titles_html)):
        result.append([titles_html[i].text, link_list[i], name_list[i], view_html[i].text])
    
    
    # soup.select를 잘 활용하여
    # 1. 신문 제목
    # 2. 기사 링크 (a tag의 href 속성) 
    # 3. 신문사명
    # 4. View
    
    return result

In [7]:
TEST_DATE_RANGE = 20 # 1년치 전체를 하는 것이 아닌 앞 20일치만 테스트 해보기 위함 (개인적으로 바꿔도 무방)

news_data = []
for date in date_string_list[:TEST_DATE_RANGE]:
     news_data.extend(get_top_news(date)) # 결과로 나온 20개를 이어 붙임

C:\Users\hyeli\anaconda3\lib\site-packages\bs4\__init__.py:203: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


In [8]:
len(news_data) # 20일 * 30개 = 600개 >>>> CORRECT!!!

600

In [9]:
news_data[:5] # 결과 예시

[['[단독] 황교안 딸 운영 사이트, 대학 진학 후 왜 문 닫았나',
  '/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=032&aid=0002952683&date=20190721&type=1&rankingSectionId=100&rankingSeq=1',
  '경향신문',
  '201,039'],
 ['조국 폭풍페북, 日주장 정면반박…"친일파" 표현은 野 반발',
  '/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=008&aid=0004251344&date=20190721&type=1&rankingSectionId=100&rankingSeq=2',
  '머니투데이',
  '185,396'],
 ['조국, 연일 對日 \'항전\' 주문…"겁먹고 쫄지말자…싸워 이겨…',
  '/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=001&aid=0010969325&date=20190721&type=1&rankingSectionId=100&rankingSeq=3',
  '연합뉴스',
  '130,198'],
 ['[김순덕의 도발]복수를 하려면 아일랜드처럼!',
  '/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=020&aid=0003230442&date=20190721&type=1&rankingSectionId=100&rankingSeq=4',
  '동아일보',
  '120,897'],
 ['조국, 또 페북에 反日 선전전..."文정부, 서희·이순신 역할…',
  '/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=023&aid=0003462170&date=20190721&type=1&rankingSectio

In [10]:
# 결과물을 데이터 프레임으로 변환 및 Column Name 부여
df_top_news = pd.DataFrame(news_data, columns=["title", "url", "press", "views"])

In [11]:
df_top_news

,title,url,press,views
0,"[단독] 황교안 딸 운영 사이트, 대학 진학 후 왜 문 닫았나",/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,경향신문,"201,039"
1,"조국 폭풍페북, 日주장 정면반박…""친일파"" 표현은 野 반발",/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,머니투데이,"185,396"
2,"조국, 연일 對日 '항전' 주문…""겁먹고 쫄지말자…싸워 이겨…",/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,연합뉴스,"130,198"
3,[김순덕의 도발]복수를 하려면 아일랜드처럼!,/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,동아일보,"120,897"
4,"조국, 또 페북에 反日 선전전...""文정부, 서희·이순신 역할…",/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,조선일보,"119,463"
...,...,...,...,...
595,"갤노트10 만져본 미국인들 ""실물 보고싶어 일부러 찾아와""",/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,서울경제,"173,703"
596,"""유튜브 뛰어드니""…월급 295만→536만원 '껑충' vs 소득 '…",/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,뉴스1,"134,014"
597,"[SNS 세상] ""공감이 최고의 위로"" 암 투병기 연재하는 뷰티…",/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,연합뉴스,"111,786"
598,네이버·카카오 2분기 성적표 '희비'…하반기 '페이전쟁' 격…,/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,뉴스1,"77,278"


In [12]:
# URL 앞에  "https://news.naver.com" 붙이기
df_top_news.url = "https://news.naver.com" + df_top_news.url 

In [13]:
df_top_news

,title,url,press,views
0,"[단독] 황교안 딸 운영 사이트, 대학 진학 후 왜 문 닫았나",https://news.naver.com/main/ranking/read.nhn?m...,경향신문,"201,039"
1,"조국 폭풍페북, 日주장 정면반박…""친일파"" 표현은 野 반발",https://news.naver.com/main/ranking/read.nhn?m...,머니투데이,"185,396"
2,"조국, 연일 對日 '항전' 주문…""겁먹고 쫄지말자…싸워 이겨…",https://news.naver.com/main/ranking/read.nhn?m...,연합뉴스,"130,198"
3,[김순덕의 도발]복수를 하려면 아일랜드처럼!,https://news.naver.com/main/ranking/read.nhn?m...,동아일보,"120,897"
4,"조국, 또 페북에 反日 선전전...""文정부, 서희·이순신 역할…",https://news.naver.com/main/ranking/read.nhn?m...,조선일보,"119,463"
...,...,...,...,...
595,"갤노트10 만져본 미국인들 ""실물 보고싶어 일부러 찾아와""",https://news.naver.com/main/ranking/read.nhn?m...,서울경제,"173,703"
596,"""유튜브 뛰어드니""…월급 295만→536만원 '껑충' vs 소득 '…",https://news.naver.com/main/ranking/read.nhn?m...,뉴스1,"134,014"
597,"[SNS 세상] ""공감이 최고의 위로"" 암 투병기 연재하는 뷰티…",https://news.naver.com/main/ranking/read.nhn?m...,연합뉴스,"111,786"
598,네이버·카카오 2분기 성적표 '희비'…하반기 '페이전쟁' 격…,https://news.naver.com/main/ranking/read.nhn?m...,뉴스1,"77,278"


## Part 2. Selenium을 이용한 크롤링

In [14]:
!pip install selenium

In [19]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()
options.headless = True # True면 창이 안뜸
driver_path = 'C://Users/hyeli/Desktop/2020/2020-2/투빅스/Week5_Crawling/chromedriver.exe' # driver 경로 설정

driver = webdriver.Chrome(driver_path) # Chrome driver 실행

In [20]:
df_top_news.url[:10]

0    https://news.naver.com/main/ranking/read.nhn?m...
1    https://news.naver.com/main/ranking/read.nhn?m...
2    https://news.naver.com/main/ranking/read.nhn?m...
3    https://news.naver.com/main/ranking/read.nhn?m...
4    https://news.naver.com/main/ranking/read.nhn?m...
5    https://news.naver.com/main/ranking/read.nhn?m...
6    https://news.naver.com/main/ranking/read.nhn?m...
7    https://news.naver.com/main/ranking/read.nhn?m...
8    https://news.naver.com/main/ranking/read.nhn?m...
9    https://news.naver.com/main/ranking/read.nhn?m...
Name: url, dtype: object

In [21]:
NEWS_TEST_RANGE = 10
for idx, news_url in enumerate(df_top_news.url[:NEWS_TEST_RANGE]):
    driver.get(df_top_news.url[idx]) # 드라이버 내에서 해당 URL로 이동
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser", from_encoding='euc-kr')
    
    # 기사 내용 크롤링
    content = soup.select_one('#articleBodyContents')
    output = ''
    for item in content.contents:
        stripped = str(item).strip()
        if stripped == "":
            continue
        if stripped[0] not in ["<", "/"]:
            output += str(item).strip('')
    output.replace('&apos;', '')

    # content 변수에 기사 내용 저장
    df_top_news.loc[idx,"content"] = output

    
    # 리액션 크롤링 및 변수에 저장
    df_top_news.loc[idx,"react_good"] = driver.find_elements_by_xpath('//*[@id="spiLayer"]/div[1]/ul/li/a/span[2]')[0].text 
    # 좋아요 투표 수
    df_top_news.loc[idx,"react_kind"] = driver.find_elements_by_xpath('//*[@id="spiLayer"]/div[1]/ul/li/a/span[2]')[1].text 
    # 훈훈해요 투표 수
    df_top_news.loc[idx,"react_sad"] = driver.find_elements_by_xpath('//*[@id="spiLayer"]/div[1]/ul/li/a/span[2]')[2].text 
    # 슬퍼요 투표 수
    df_top_news.loc[idx,"react_upset"] = driver.find_elements_by_xpath('//*[@id="spiLayer"]/div[1]/ul/li/a/span[2]')[3].text 
    # 화나요 투표 수
    df_top_news.loc[idx,"react_want"] = driver.find_elements_by_xpath('//*[@id="spiLayer"]/div[1]/ul/li/a/span[2]')[4].text 
    # 후속기사원해요 투표 수

In [22]:
df_top_news.head(5)

,title,url,press,views,content,react_good,react_kind,react_sad,react_upset,react_want
0,"[단독] 황교안 딸 운영 사이트, 대학 진학 후 왜 문 닫았나",https://news.naver.com/main/ranking/read.nhn?m...,경향신문,"201,039",본문 내용 TV플레이어 “대체로 다 그랬던 것 같다. 봉사 스펙으로 대학에 가는...,131,16,17,"2,605",121
1,"조국 폭풍페북, 日주장 정면반박…""친일파"" 표현은 野 반발",https://news.naver.com/main/ranking/read.nhn?m...,머니투데이,"185,396","본문 내용 TV플레이어 \n\t\n\t[머니투데이 김성휘 ,백지수 기자] [[...","1,710",32,13,"6,058",48
2,"조국, 연일 對日 '항전' 주문…""겁먹고 쫄지말자…싸워 이겨…",https://news.naver.com/main/ranking/read.nhn?m...,연합뉴스,"130,198","본문 내용 TV플레이어 \n\t\n\t""文정부, 서희와 이순신 역할 함께 수행…...","1,799",19,13,"9,120",30
3,[김순덕의 도발]복수를 하려면 아일랜드처럼!,https://news.naver.com/main/ranking/read.nhn?m...,동아일보,"120,897",본문 내용 TV플레이어 \n\t\n\t친일잔재를 청산하고 한번도 경험하지 못한 ...,"3,017",19,16,623,37
4,"조국, 또 페북에 反日 선전전...""文정부, 서희·이순신 역할…",https://news.naver.com/main/ranking/read.nhn?m...,조선일보,"119,463","본문 내용 TV플레이어 조국 청와대 민정수석은 21일 ""문재인 정부는 국익 수호...",374,13,14,"11,468",38


In [23]:
for i in range(df_top_news.shape[0]): 
    df_top_news['content'][i] = str(df_top_news['content'][i]).replace(' 본문 내용  TV플레이어 ', '').strip()

In [24]:
df_top_news.head()

,title,url,press,views,content,react_good,react_kind,react_sad,react_upset,react_want
0,"[단독] 황교안 딸 운영 사이트, 대학 진학 후 왜 문 닫았나",https://news.naver.com/main/ranking/read.nhn?m...,경향신문,"201,039",“대체로 다 그랬던 것 같다. 봉사 스펙으로 대학에 가는 경우는 외고 같은 데에서도...,131,16,17,"2,605",121
1,"조국 폭풍페북, 日주장 정면반박…""친일파"" 표현은 野 반발",https://news.naver.com/main/ranking/read.nhn?m...,머니투데이,"185,396","[머니투데이 김성휘 ,백지수 기자] [[300]징용판결 해설 글 ""대통령 법률보좌...","1,710",32,13,"6,058",48
2,"조국, 연일 對日 '항전' 주문…""겁먹고 쫄지말자…싸워 이겨…",https://news.naver.com/main/ranking/read.nhn?m...,연합뉴스,"130,198","""文정부, 서희와 이순신 역할 함께 수행…지레 겁먹지 말아야""""文정부 매도 정치인·...","1,799",19,13,"9,120",30
3,[김순덕의 도발]복수를 하려면 아일랜드처럼!,https://news.naver.com/main/ranking/read.nhn?m...,동아일보,"120,897","친일잔재를 청산하고 한번도 경험하지 못한 나라로 가는 것이 목적이라면, 문재인 정부...","3,017",19,16,623,37
4,"조국, 또 페북에 反日 선전전...""文정부, 서희·이순신 역할…",https://news.naver.com/main/ranking/read.nhn?m...,조선일보,"119,463","조국 청와대 민정수석은 21일 ""문재인 정부는 국익 수호를 위해 '서희'의 역할과 ...",374,13,14,"11,468",38


# 데이터 저장

> 파일 형태로 크롤링한 데이터를 저장

In [25]:
df_top_news.to_csv('Crawling_assignment.csv', mode='w', encoding='utf-8-sig')